# cPEPmatch

Recent updates: 

- Supports all motif size selections (4-6 work best). 
- Allows a selection of non-consecutive amino acids (set "conscutive = False").
- The database program creator is now attached within the same program (if you want to run it, set "CREATE_CYCLOLIB=1")


Created by Brianda L. Santini - 
Physics Department T38, Technical University of Munich, Garching, Germany.

In [6]:
import sys,os,re, glob
import operator #set of efficient functions
import pickle #store data on disc
import time
import warnings
import numpy as np
import shutil #import copyfile
import csv

from scipy import spatial
from glob import glob
from collections import defaultdict
from itertools import product, combinations, permutations
from sys import exit

from modeller import *
from modeller.optimizers import MolecularDynamics, ConjugateGradients
from modeller.automodel import autosched

import Bio
from Bio.PDB import PDBParser
from Bio.PDB.parse_pdb_header import parse_pdb_header

import vmd
from vmd import molecule, atomsel

import MDAnalysis as mda
import nglview as nv
import pytraj as pt

In [30]:
CREATE_INTERFACES=1
PROCESS_INTERFACE=1
FIND_MATCH=1
CREATE_CYCLOLIB=1
MUTATE=1

interface_cutoff = 7
frmsd_threshold = 0.7
motif_size = 4 #can support all motif sizes (4-7 work best) 
consecutive = True #True or False


protein='1 to 418' #Resid selection of protein to mimic.
target = '419 to 424' #Resid selection of protein to target.
pdb_name = '1e03'


my_location = '/home/brianda/Dev/cPEPmatch/' #EXAMPLE: '/home/margrethe/Documents/cPEPmatch/'
database_location = "/cyclo_database/"
os.chdir('{}{}/' .format(my_location, pdb_name))


#### Cyclo Database

Characterization the cyclic peptide motifs.

In [31]:
def int_list(lyst, sep=' ', quote=None):
    """Return the list of numbers in 'lyst' as a string separated by 'sep'."""
    f = str if quote is None else lambda i: quote+str(i)+quote
    return sep.join(map(f, lyst))

def cyclo_distance_matrix(nres, R, fname):
    M = dict()
    combination = combinations(R, nres)
    #residue_combination = list(combination)
    #print(len(combination))
    count = 1
    for i in combination:
        resids = i
        #if not np.all(np.diff(resids)==1): continue
        rlist = int_list(resids, quote='"')
        #if not rnbrs: continue
        QA = '''(chain M and name CA and (altloc "A" or altloc "") and resid %(rlist)s)'''
        QA = ' '.join(str(QA).split())
        
        resid_query = atomsel(QA % dict(rlist=rlist))
        r = np.array(resid_query.centerperresidue())
        # Ignore sequences that are either near the ends of a chain, or are near ends of a broken chain
        if len(r) != nres: continue
        #assert len(r) == nres, (len(r), r, nres, res, rnbrs, resid_query, fname)
        dr = spatial.distance_matrix(r,r)
        m = dr[np.triu_indices(nres,k=1)]
        M[str(count)] = dict(m=m, resid=resids)
        count = count + 1
    return M

def cyclo_distance_matrix_consecutive(nres, R, fname):
    M = dict()
    
    for i,res in enumerate(R[:-nres+1]):
        #print(res, res+nres, R[i:i+nres])
        resids = R[i:i+nres]
        if not np.all(np.diff(resids)==1): continue
        rlist = int_list(resids, quote='"')
        #if not rnbrs: continue
        QA = '''(chain M and name CA and (altloc "A" or altloc "") and resid %(rlist)s)'''
        QA = ' '.join(str(QA).split())
        
        resid_query = atomsel(QA % dict(rlist=rlist))
        r = np.array(resid_query.centerperresidue())
        
        #
        # Ignore sequences that are either near the ends of a chain, or are near ends of a broken chain
        #
        if len(r) != nres: continue
        assert len(r) == nres, (len(r), r, nres, res, rnbrs, resid_query, fname)
        
        dr = spatial.distance_matrix(r,r)
        m = dr[np.triu_indices(nres,k=2)]
        M[resids[0]] = dict(
            m=m,
#            r=r,
            resid=resids,
        )
    return M


def cyclo_motifs(molid):
    MC = dict()
    C = atomsel('chain M and protein and backbone and (altloc "A" or altloc "") ')
    C_res = list(sorted(set(C.resid)))
    if consecutive:
        MC = cyclo_distance_matrix_consecutive(motif_size, C_res, cyclo)
    else:
        MC = cyclo_distance_matrix(motif_size, C_res, cyclo)
    return (MC)


In [32]:
if CREATE_CYCLOLIB:
    os.chdir(my_location + database_location)
    #os.chdir('/home/brianda/Documents/cPEPmatch/cyclo_database')

    parser = PDBParser()
    cyclo_mtfs = []
    
    if MUTATE:
        lib = ['1cnl', '1ebp', '1foz', '1jbl', '1l5g', '1npo', '1qx9', '1sld', '1sle', '1urc', '1vwb','2ajw', 
               '2ak0', '2c9t', '2jrw', '2lws', '2lwt', '2lwu', '2lwv', '2m2g', '2mgo', '2n07', '2ox2', '3av9', 
               '3ava', '3avb', '3avi', '3p8f', '3wbn', '3wne', '3wnf', '3zwz', '4gly', '4ib5', 
               '4k8y', '4kts', '4ktu', '4mq9', '5bxo', '5eoc', '5glh', '5lff', '5lso', 
               '5vav', '5xco', '5xn3', '6awk', '6awm', '6axi', '6pin', '6pio', '6pip' 
              ]
    else:
        lib = ['2c9t', '1cnl', '1tk2', '5eoc', '3zwz', '4z0f', '4zjx', '4ib5', '3oe0', '6c1r', '1urc', '4w4z', 
              '1ebp', '3pqz', '3avb', '4n7y', '5xco', '4k8y', '1jbl', '3p8f', '4i80', '5afg', '3wbn', '1npo', 
              '2mgo', '3wmg', '5kgn', '5b4w', '5lso', '5xn3', '1sld', '5bxo', '4zqw', '4ktu', '4kts', '4twt', 
              '3qn7', '3av9', '3ava', '3avi', '3wnf', '1sle', '1vwb', '5n99', '4os1', '4mnw', '4jk5', '4gly', 
              '4x1n', '5glh', '2m2g', '2lwv', '6pip', '2lwt', '1foz', '2ajw', '5lff', '2ox2', '6axi', '2lwu', 
              '3wne', '2ak0', '2lws', '1qx9', '6pin', '2n07', '1l5g', '6pio', '2jrw', '6awm', '5vav', '6awk' ]
    

    
    for c in lib:
        motifs = dict()
        cyclo = ('%s-cp.pdb' % c)
        molid = molecule.load("pdb", cyclo )
        c_mtfs = cyclo_motifs(molid)
        if c_mtfs is not None:
            #save_interacting_sidechains(os.path.splitext(f)[0], isc)
            motifs.update({c: c_mtfs})
            cyclo_mtfs.append(motifs)
            
    database = open("database_%s.pkl" % motif_size, "wb")
    pickle.dump(cyclo_mtfs, database)
    database.close()


# Protein-Target characterization

This section selects the residues of the PPI interface and characterizes its CA motifs. 
Name the PPI input file as 'complex.pdb'

## 1. Interface 

In [33]:
def interfaces(molid, cutoff=interface_cutoff, minresidues=4):
    I = []
    I = atomsel("resid {} and same residue as exwithin {} of resid {}".format(protein, cutoff, target))
    if len(set(I.residue)) < minresidues: 
        I = []
    return I


def create_interfaces(): 
    parser = PDBParser()
    try:
        pdb_fname = ("{}.pdb" .format(pdb_name))
        molid  = molecule.load("pdb", pdb_fname) #from vmd library 
        Iall = interfaces(molid) 
        int_fname = 'interface.pdb'
        print('%s -> %s' % (pdb_fname, int_fname))
        molecule.write(molid, 'pdb', int_fname, selection=Iall)
        molecule.delete(molid)
    except RuntimeError as e:
        warn(e)
        pass


In [34]:
if CREATE_INTERFACES:
    os.chdir('{}{}/' .format(my_location, pdb_name))
    create_interfaces()

1e03.pdb -> interface.pdb


In [35]:
view = nv.show_file('{}{}/interface.pdb' .format(my_location, pdb_name))
view.clear_representations()
view.add_representation('ball+stick', selection='protein')
view

NGLWidget()

## 2. Motif construction

In [36]:
def int_list(lyst, sep=' ', quote=None):
    """Return the list of numbers in 'lyst' as a string separated by 'sep'."""
    f = str if quote is None else lambda i: quote+str(i)+quote
    return sep.join(map(f, lyst))

def load_interface(pdb_fname): # process_all_interfaces(load_interface,...)
    """Return a vmd molecule for the interface found in pdb_fname in a tuple of (vmd-mol, pdb-code, tag, list-of-chains)."""
    d,f = os.path.split(pdb_fname) # d = path/.../, f = x.pdb
    pdb,tag,chains,ext = f.split('.') # e.g. 4gko, pp, B-G, pdb
    chains = chains.split('-')
    mol = molecule.load("pdb", pdb_fname) # What is molecule in that context? 
    #vmd.render.render('PostScript', pdb_fname + '.ps')
    #molecule.ssrecalc(int(mol)) # recalculate secondary structure
    return (mol, pdb, tag, chains) # (12, '4gko', 'pp', ['D', 'J'])


def distance_matrix_consecutive(nres, R, fname):
    M = dict()
    for i,res in enumerate(R[:-nres+1]):
        resids = R[i:i+nres]
        if not np.all(np.diff(resids)==1): continue
        rlist = int_list(resids, quote='"')      
        resid_query = atomsel("name CA and resid {}" .format(rlist))
        clz = resid_query.structure
        r = np.array(resid_query.centerperresidue())
        
        # Ignore sequences that are either near the ends of a chain, or are near ends of a broken chain
        
        if len(r) != nres: continue
        assert len(r) == nres, (len(r), r, nres, res, rnbrs, resid_query, fname)
        
        dr = spatial.distance_matrix(r,r)
        m = dr[np.triu_indices(nres,k=2)]
        M[resids[0]] = dict(
            m=m,
            resid=resids,
        )
    return M


def distance_matrix(nres, R, fname):
    M = dict()
    combination = combinations(R, nres)
    #residue_combination = list(combination)
    count = 1
    for i in combination:
        resids = i
        #if not np.all(np.diff(resids)==1): continue
        rlist = int_list(resids, quote='"')
        #if not rnbrs: continue
        resid_query = atomsel("name CA and resid {}" .format(rlist))
        r = np.array(resid_query.centerperresidue())
        # Ignore sequences that are either near the ends of a chain, or are near ends of a broken chain
        #if len(r) != nres: continue
        #assert len(r) == nres, (len(r), r, nres, res, rnbrs, resid_query, fname)
        dr = spatial.distance_matrix(r,r)
        m = dr[np.triu_indices(nres,k=1)]
        M[str(count)] = dict(m=m, resid=resids)
        count = count + 1
    return M
        
def motifs(molid):
    P = atomsel("all")
    P_res = list(sorted(set(P.resid)))
    if consecutive:
        MP = distance_matrix_consecutive(motif_size, P_res, inter_fname)
    else:
        MP = distance_matrix(motif_size, P_res, inter_fname)
    return (MP)


In [37]:
if PROCESS_INTERFACE:
    os.chdir('{}{}/'.format(my_location, pdb_name))
    parser = PDBParser()
    inter_fname = "interface.pdb"
    molid  = molecule.load("pdb", inter_fname)
    mtfs = motifs(molid)
    
mtfs

{3: {'m': array([5.38666371, 7.81653292, 6.73579162]), 'resid': [3, 4, 5, 6]},
 4: {'m': array([ 6.73579162, 10.28086966,  7.04468341]),
  'resid': [4, 5, 6, 7]},
 5: {'m': array([7.04468341, 8.36494192, 5.68353729]), 'resid': [5, 6, 7, 8]},
 6: {'m': array([5.68353729, 5.15543138, 5.44578758]), 'resid': [6, 7, 8, 9]},
 7: {'m': array([5.44578758, 6.22028731, 5.43225326]), 'resid': [7, 8, 9, 10]},
 27: {'m': array([6.64064317, 8.87923684, 6.6118616 ]),
  'resid': [27, 28, 29, 30]},
 28: {'m': array([6.6118616 , 5.95431307, 5.5173707 ]),
  'resid': [28, 29, 30, 31]},
 29: {'m': array([5.5173707 , 9.07257982, 7.03192473]),
  'resid': [29, 30, 31, 32]},
 30: {'m': array([7.03192473, 9.41077431, 6.21125978]),
  'resid': [30, 31, 32, 33]},
 31: {'m': array([6.21125978, 8.7194961 , 5.42618795]),
  'resid': [31, 32, 33, 34]},
 32: {'m': array([5.42618795, 5.08121134, 5.41419375]),
  'resid': [32, 33, 34, 35]},
 33: {'m': array([5.41419375, 4.98696946, 5.37523153]),
  'resid': [33, 34, 35, 36]

## 3. Backbone Match

This section matches the CA distance motifs of the PPI and the cyclic peptide database. 18:35

In [38]:
cyclo_mtfs = [ ]

with open("{}{}database_{}.pkl" .format(my_location,database_location, motif_size),'rb') as database:
    cyclo_mtfs = pickle.load(database)
    database_length = len(cyclo_mtfs)

if FIND_MATCH:
    match = dict()
    all_match = []
    matches = []
    d_ppi = []
    
    for pp in mtfs:
        m_count = len(mtfs[pp]["m"])
        d_ppi = []
        for j in range(0, m_count):
            d_ppi_single = mtfs[pp]["m"][j]
            d_ppi.append(d_ppi_single)
        for i in range(0,database_length):
            for c in cyclo_mtfs[i]:
                peptide = c
                for d in (cyclo_mtfs[i][c]):
                    d_cyclo = []
                    for k in range(0, m_count):
                        d_cyclo_single = cyclo_mtfs[i][c][d]["m"][k]
                        d_cyclo.append(d_cyclo_single)
                    sum = 0
                    for r in range(0, m_count):
                        dd = (d_ppi[r]-d_cyclo[r])**2
                        sum = sum + dd
                    rmsd = np.sqrt(sum)
                    if rmsd < frmsd_threshold:
                        rmsd = rmsd.round(decimals=4)
                        match = ( peptide , rmsd,  cyclo_mtfs[i][c][d]["resid"], mtfs[pp]["resid"] )
                        all_match.append(match) 


all_match.sort()
all_match



[('1cnl', 0.3671, [9, 10, 11, 12], [7, 8, 9, 10]),
 ('1cnl', 0.3912, [1, 2, 3, 4], [33, 34, 35, 36]),
 ('1cnl', 0.4108, [1, 2, 3, 4], [6, 7, 8, 9]),
 ('1cnl', 0.4194, [1, 2, 3, 4], [100, 101, 102, 103]),
 ('1cnl', 0.4492, [1, 2, 3, 4], [32, 33, 34, 35]),
 ('1cnl', 0.4605, [5, 6, 7, 8], [7, 8, 9, 10]),
 ('1cnl', 0.4733, [7, 8, 9, 10], [31, 32, 33, 34]),
 ('1cnl', 0.4818, [4, 5, 6, 7], [31, 32, 33, 34]),
 ('1cnl', 0.5807, [1, 2, 3, 4], [102, 103, 104, 105]),
 ('1cnl', 0.5848, [8, 9, 10, 11], [31, 32, 33, 34]),
 ('1cnl', 0.627, [6, 7, 8, 9], [102, 103, 104, 105]),
 ('1cnl', 0.6417, [1, 2, 3, 4], [101, 102, 103, 104]),
 ('1cnl', 0.659, [4, 5, 6, 7], [5, 6, 7, 8]),
 ('1cnl', 0.6857, [1, 2, 3, 4], [99, 100, 101, 102]),
 ('1cnl', 0.6984, [2, 3, 4, 5], [7, 8, 9, 10]),
 ('1ebp', 0.3303, [13, 14, 15, 16], [30, 31, 32, 33]),
 ('1ebp', 0.3828, [1, 2, 3, 4], [30, 31, 32, 33]),
 ('1ebp', 0.4104, [12, 13, 14, 15], [4, 5, 6, 7]),
 ('1ebp', 0.4519, [2, 3, 4, 5], [4, 5, 6, 7]),
 ('1ebp', 0.4742, [7, 8, 

## 4. Superimpose & Mutate

This section superimposes the matched structures. Output files are the superimposed pdb's and a full list of the matches.

In [39]:
#MODELLER optimization

def optimize(atmsel, sched):
    #conjugate gradient
    for step in sched:
        step.optimize(atmsel, max_iterations=200, min_atom_shift=0.001)
    #md
    refine(atmsel)
    cg = ConjugateGradients()
    cg.optimize(atmsel, max_iterations=200, min_atom_shift=0.001)


#molecular dynamics
def refine(atmsel):
    # at T=1000, max_atom_shift for 4fs is cca 0.15 A.
    md = MolecularDynamics(cap_atom_shift=0.39, md_time_step=4.0,
                           md_return='FINAL')
    init_vel = True
    for (its, equil, temps) in ((200, 20, (150.0, 250.0, 400.0, 700.0, 1000.0)),
                                (200, 600,
                                 (1000.0, 800.0, 600.0, 500.0, 400.0, 300.0))):
        for temp in temps:
            md.optimize(atmsel, init_velocities=init_vel, temperature=temp,
                         max_iterations=its, equilibrate=equil)
            init_vel = False


#use homologs and dihedral library for dihedral angle restraints
def make_restraints(mdl1, aln):
   rsr = mdl1.restraints
   rsr.clear()
   s = Selection(mdl1)
   for typ in ('stereo', 'phi-psi_binormal'):
       rsr.make(s, restraint_type=typ, aln=aln, spline_on_site=True)
   for typ in ('omega', 'chi1', 'chi2', 'chi3', 'chi4'):
       rsr.make(s, restraint_type=typ+'_dihedral', spline_range=4.0,
                spline_dx=0.3, spline_min_points = 5, aln=aln,
                spline_on_site=True)
    
    
    
def overlap_finder(match_path, complex_fname, matched_sequence):
    
    parser = PDBParser()
    ppi = molecule.load("pdb", complex_fname)    
    receptor_chain = atomsel("resid {}" .format(target)) #change this if the receptor has a different chain name.
    #receptor_chain = ' '.join(str(receptor_chain).split())
    
    parser = PDBParser()
    cyclo = molecule.load("pdb", (match_path + 'cpep_aligned.pdb'))
    cyclo_chain = atomsel('chain M and not resid {}'.format(matched_sequence))
    #cyclo_chain = ' '.join(str(cyclo_chain).split())
    
    overlap = atomsel.contacts(cyclo_chain, receptor_chain, 1)
    
    return overlap         

In [ ]:
#TODO: Adapt a 5-motif superimposition and mutation

os.chdir('{}{}'.format(my_location, pdb_name))
cwd = ('{}{}'.format(my_location, pdb_name))

#Delete old match files 
dir = [os.listdir('.')]
for f in dir:
    for a in f:
        if "match" in a:
            try:
                os.remove(a)
            except OSError as e:
                 pass
                    
                    

match_directory_list = []
all_mutations = dict()
motif_end = motif_size - 1

cr = 1
c1 = 1
real_match_count = 1

with open('match_list.txt', 'w') as f:
    f.write("List of matches found by cPEPmatch.\nMotif Size: {} \nInterface Cutoff: {} \nFit-RMSD Threshold: {} \n\n" .format(motif_size, interface_cutoff,frmsd_threshold))
    f.write(" Match  pdb   Dist-RMSD    CPep Residues        PP-Interface Residues   Fit-RMSD\n\n")
    
    for match in all_match: 
        
        no_overlap_match = []
    
        pdb_parser = Bio.PDB.PDBParser(QUIET = True)
        matched_cyclopep = match[0]
        match_dir = ("match{}_{}".format(cr, matched_cyclopep))
        match_directory_list.append(match_dir)
    
        
        #Selection of residues to be aligned
    
        #cyclo_start_id = match[2][0]
        #cyclo_end_id = match[2][motif_end]
        cyclo_residues_to_be_aligned = match[2]
    
        #ppi_start_id = match[3][0]
        #ppi_end_id = match[3][motif_end]
        ppi_residues_to_be_aligned = match[3]
    
    
        #Create a directory for each match
    
        try:
            os.mkdir("{}".format(match_dir))
            cr  =  cr + 1
        except:
            print("Match folder already exist")
    
        #cpep = (cwd + ('/cyclo_library/{}-cp.pdb'.format(matched_cyclopep)))
        cpep = ('{}cyclo_database/{}-cp.pdb'.format(my_location, matched_cyclopep))
        match_folder = (cwd + ('/{}/cpep_match.pdb'.format(match_dir)))
        match_path = (cwd + ('/{}/'.format(match_dir)))

        try:
            #os.popen('cp {} {}'.format(cpep, os.path.join(match_folder, filename)))
            #copyfile(cpep, match_folder)
            shutil.copy(cpep, match_folder, follow_symlinks=True)
            
        except IOError as e:
            print("Unable to copy file. %s" % e)
            
        
        # Get the structures
        cyclo_structure = pdb_parser.get_structure("reference", (cwd + ("/{}/cpep_match.pdb".format(match_dir))))  
        ppi_structure = pdb_parser.get_structure("sample", (cwd + "/interface.pdb"))
        
        
        cyclo_model = cyclo_structure[0]
        cyclo_chain = cyclo_model['M']
        ppi_model = ppi_structure[0]
        ppi_chain = ppi_model['X']
        

        # List of atoms to align.
        cyclo_atoms = []
        ppi_atoms = []

        for cyclo_chain in cyclo_model:
            for cyclo_res in cyclo_chain:
                if cyclo_res.get_id()[1] in cyclo_residues_to_be_aligned: # Check if residue number ( .get_id() ) is in the list
                    cyclo_atoms.append(cyclo_res['CA'])

        
        for ppi_chain in ppi_model:
            for ppi_res in ppi_chain:
                if ppi_res.get_id()[1] in ppi_residues_to_be_aligned:
                    ppi_atoms.append(ppi_res['CA'])
        
        # Superimposer:
        super_imposer = Bio.PDB.Superimposer()
        super_imposer.set_atoms(ppi_atoms, cyclo_atoms)
        super_imposer.apply(cyclo_model.get_atoms())

        # Add to fit_RMSD the results:
        superimpose_rmsd = super_imposer.rms
        lst = list(match)
        lst = lst + [superimpose_rmsd]
        match = tuple(lst)
    
        # Save the aligned version 
        io = Bio.PDB.PDBIO()
        io.set_structure(cyclo_structure) 
        io.save(match_path + "cpep_aligned.pdb") 
        
        
        #Eliminates the matched cyclic peptides that overlap with the receptor. 
        
        cyclo_rlist = int_list(match[2], quote='"')
        matched_cyclo_residues = cyclo_rlist
        overlap = overlap_finder(match_path, (cwd + ("/{}.pdb".format(pdb_name))), matched_cyclo_residues)

        if overlap == [[],[]]:
            no_overlap_match = match
        
        
        #WRITE OUTPUT FILE
        if no_overlap_match != []:
            c2 = 0
            f.write("{:>6}".format(c1))
            for i in no_overlap_match:
                if c2 < 1:
                    f.write("{:>6} ".format(i))
                elif c2 < 2:
                    f.write("{:>9.4f}".format(i))
                elif c2 < 3:
                    f.write("   ")
                    for r in i:
                        f.write("{:>5}".format(r))
                elif c2 < 4:
                    f.write("   ")
                    for r in i:
                        f.write("{:>5}".format(r))
                else:
                    f.write("   ")
                    f.write("{:>9.4f}".format(i))

                c2 = c2 + 1

            f.write("\n")
            c1 = c1 + 1  

    
     #MUTATE
            if MUTATE:
           
                modelname = (match_path + 'cpep_aligned.pdb') 
                motif_length = len(match[2]) - 1
                mutation_count = 0

                for matched_residue in match[2]:

                    respos = str(matched_residue)
                    residue_number_to_match = match[3][mutation_count]
                    restyp = ppi_chain[residue_number_to_match].get_resname()     
                    cyclic_restyp = cyclo_chain[matched_residue].get_resname()
                    chain = 'M'
                    disulfide = 'CYX'

                    if cyclic_restyp != disulfide:

                        try:
                            log.verbose()

                            # Set a different value for rand_seed to get a different final model
                            env = Environ(rand_seed=-49837)

                            env.io.hetatm = True
                            #soft sphere potential
                            env.edat.dynamic_sphere=False
                            #lennard-jones potential (more accurate)
                            env.edat.dynamic_lennard=True
                            env.edat.contact_shell = 4.0
                            env.edat.update_dynamic = 0.39

                            # Read customized topology file with phosphoserines (or standard one)
                            env.libs.topology.read(file='$(LIB)/top_heav.lib')

                            # Read customized CHARMM parameter library with phosphoserines (or standard one)
                            env.libs.parameters.read(file='$(LIB)/par.lib')


                            # Read the original PDB file and copy its sequence to the alignment array:
                            mdl1 = Model(env, file=modelname)
                            ali = Alignment(env)
                            ali.append_model(mdl1, atom_files=modelname, align_codes=modelname)

                            #set up the mutate residue selection segment
                            s = Selection(mdl1.chains[chain].residues[respos])

                            #perform the mutate residue operation
                            s.mutate(residue_type=restyp)
                            #get two copies of the sequence.  A modeller trick to get things set up
                            ali.append_model(mdl1, align_codes=modelname)

                            # Generate molecular topology for mutant
                            mdl1.clear_topology()
                            mdl1.generate_topology(ali[-1])


                            # Transfer all the coordinates you can from the template native structure
                            # to the mutant (this works even if the order of atoms in the native PDB
                            # file is not standard):
                            #here we are generating the model by reading the template coordinates
                            mdl1.transfer_xyz(ali)

                            # Build the remaining unknown coordinates
                            mdl1.build(initialize_xyz=False, build_method='INTERNAL_COORDINATES')

                            #yes model2 is the same file as model1.  It's a modeller trick.
                            mdl2 = Model(env, file=modelname)

                            #required to do a transfer_res_numb
                            #ali.append_model(mdl2, atom_files=modelname, align_codes=modelname)
                            #transfers from "model 2" to "model 1"
                            mdl1.res_num_from(mdl2,ali)

                            #It is usually necessary to write the mutated sequence out and read it in
                            #before proceeding, because not all sequence related information about MODEL
                            #is changed by this command (e.g., internal coordinates, charges, and atom
                            #types and radii are not updated).

                            mdl1.write(file=modelname+restyp+respos+'.tmp')
                            mdl1.read(file=modelname+restyp+respos+'.tmp')

                            #set up restraints before computing energy
                            #we do this a second time because the model has been written out and read in,
                            #clearing the previously set restraints
                            make_restraints(mdl1, ali)

                            #a non-bonded pair has to have at least as many selected atoms
                            mdl1.env.edat.nonbonded_sel_atoms=1

                            sched = autosched.loop.make_for_model(mdl1)

                            #only optimize the selected residue (in first pass, just atoms in selected
                            #residue, in second pass, include nonbonded neighboring atoms)
                            #set up the mutate residue selection segment
                            s = Selection(mdl1.chains[chain].residues[respos])

                            mdl1.restraints.unpick_all()
                            mdl1.restraints.pick(s)

                            try:
                                s.energy()
                            except RuntimeError:
                                pass

                            s.randomize_xyz(deviation=4.0)

                            mdl1.env.edat.nonbonded_sel_atoms=2

                            try:
                                optimize(s, sched)
                            except RuntimeError:
                                pass


                            #feels environment (energy computed on pairs that have at least one member
                            #in the selected)
                            mdl1.env.edat.nonbonded_sel_atoms=1

                            try:
                                optimize(s, sched)
                            except RuntimeError:
                                pass


                            try:
                                s.energy()
                            except RuntimeError:
                                pass


                            #Output file
                            #mdl1.write(file=modelname+restyp+respos+'.pdb')
                            mdl1.write(match_path + ('mutation{}.pdb'.format(mutation_count)))

                            #delete the temporary file
                            os.remove(modelname+restyp+respos+'.tmp')

                            shutil.copyfile((match_path + ('mutation{}.pdb'.format(mutation_count))), (match_path + ('mutated_match.pdb')))
                            modelname = (match_path + ('mutated_match.pdb'))

                            mutation_count = mutation_count + 1


                        except RuntimeError as e:
                            print('Non-standard amino acid found in the cyclic peptide sequence {}.'
                                  ' I will not be mutated'.format(matched_cyclopep))
                            pass

                    else:
                        mutation_count = mutation_count + 1
            
            
                shutil.copyfile((match_path + ('mutated_match.pdb')), 
                                ('match{}_{}.pdb'.format(real_match_count, matched_cyclopep)))
                real_match_count = real_match_count + 1
            
        
                for fname in os.listdir(match_path):
                    if fname.startswith("mutation"):
                        os.remove(os.path.join(match_path, fname))
            
                shutil.rmtree(match_dir)    
            
            
        else:
            shutil.rmtree(match_dir)


                


Match folder already exist

                         MODELLER 10.1, 2021/03/12, r12156

     PROTEIN STRUCTURE MODELLING BY SATISFACTION OF SPATIAL RESTRAINTS


                     Copyright(c) 1989-2021 Andrej Sali
                            All Rights Reserved

                             Written by A. Sali
                               with help from
              B. Webb, M.S. Madhusudhan, M-Y. Shen, G.Q. Dong,
          M.A. Marti-Renom, N. Eswar, F. Alber, M. Topf, B. Oliva,
             A. Fiser, R. Sanchez, B. Yerkovich, A. Badretdinov,
                     F. Melo, J.P. Overington, E. Feyfant
                 University of California, San Francisco, USA
                    Rockefeller University, New York, USA
                      Harvard University, Cambridge, USA
                   Imperial Cancer Research Fund, London, UK
              Birkbeck College, University of London, London, UK


Kind, OS, HostName, Kernel, Processor: 4, Linux gecko 5.3.0-51-generic x86_64
Date

openf___224_> Open           ${MODINSTALL10v1}/modlib/mnch3.lib
openf___224_> Open           ${MODINSTALL10v1}/modlib/xs4.mat
rdrrwgh_268_> Number of residue types:       21
openf___224_> Open           $(LIB)/top_heav.lib
read_to_681_> topology.submodel read from topology file:        3
openf___224_> Open           $(LIB)/par.lib

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1333938    1302.674     1.272

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1335538    1304.236     1.274

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1339518    1308.123     1.277

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1341518    1310.076     1.279

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1344518    1313.006     1.282

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1349018    1317.400     1.287

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1354590    1322.84

openf___224_> Open           ${MODINSTALL10v1}/modlib/mnch3.lib
openf___224_> Open           ${MODINSTALL10v1}/modlib/xs4.mat
rdrrwgh_268_> Number of residue types:       21
openf___224_> Open           $(LIB)/top_heav.lib
read_to_681_> topology.submodel read from topology file:        3
openf___224_> Open           $(LIB)/par.lib

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1332588    1301.355     1.271

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1334188    1302.918     1.272

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1338168    1306.805     1.276

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1340168    1308.758     1.278

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1343168    1311.688     1.281

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1347668    1316.082     1.285

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1353240    1321.52



>> ENERGY; Differences between the model's features and restraints:
Number of all residues in MODEL                   :       13
Number of all, selected real atoms                :       88       8
Number of all, selected pseudo atoms              :        0       0
Number of all static, selected restraints         :      411      50
COVALENT_CYS                                      :        F
NONBONDED_SEL_ATOMS                               :        1
Number of non-bonded pairs (excluding 1-2,1-3,1-4):       18
Dynamic pairs routine                             : 5, NATM_SEL x NATM double loop
Atomic shift for contacts update (UPDATE_DYNAMIC) :    0.390
LENNARD_JONES_SWITCH                              :    6.500   7.500
COULOMB_JONES_SWITCH                              :    6.500   7.500
RESIDUE_SPAN_RANGE                                :        0   99999
NLOGN_USE                                         :       15
CONTACT_SHELL                                     :    4.000
DYNAMI

openf___224_> Open           ${MODINSTALL10v1}/modlib/xs4.mat
rdrrwgh_268_> Number of residue types:       21
openf___224_> Open           $(LIB)/top_heav.lib
read_to_681_> topology.submodel read from topology file:        3
openf___224_> Open           $(LIB)/par.lib

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1331340    1300.137     1.270

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1332940    1301.699     1.271

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1336920    1305.586     1.275

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1338920    1307.539     1.277

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1341920    1310.469     1.280

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1346420    1314.863     1.284

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1351992    1320.305     1.289

Dynamically allocated memory at  amaxparameters [B,

...

openf___224_> Open           ${MODINSTALL10v1}/modlib/xs4.mat
rdrrwgh_268_> Number of residue types:       21
openf___224_> Open           $(LIB)/top_heav.lib
read_to_681_> topology.submodel read from topology file:        3
openf___224_> Open           $(LIB)/par.lib

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1380348    1347.996     1.316

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1381948    1349.559     1.318

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1385928    1353.445     1.322

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1387928    1355.398     1.324

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1390928    1358.328     1.326

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1395428    1362.723     1.331

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1401000    1368.164     1.336

Dynamically allocated memory at  amaxparameters [B,

openf___224_> Open           ${MODINSTALL10v1}/modlib/mnch3.lib
openf___224_> Open           ${MODINSTALL10v1}/modlib/xs4.mat
rdrrwgh_268_> Number of residue types:       21
openf___224_> Open           $(LIB)/top_heav.lib
read_to_681_> topology.submodel read from topology file:        3
openf___224_> Open           $(LIB)/par.lib

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1309414    1278.725     1.249

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1311014    1280.287     1.250

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1314994    1284.174     1.254

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1316994    1286.127     1.256

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1319994    1289.057     1.259

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1324494    1293.451     1.263

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1330066    1298.89

openf___224_> Open           ${MODINSTALL10v1}/modlib/mnch3.lib
openf___224_> Open           ${MODINSTALL10v1}/modlib/xs4.mat
rdrrwgh_268_> Number of residue types:       21
openf___224_> Open           $(LIB)/top_heav.lib
read_to_681_> topology.submodel read from topology file:        3
openf___224_> Open           $(LIB)/par.lib

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1309184    1278.500     1.249

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1310784    1280.062     1.250

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1314764    1283.949     1.254

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1316764    1285.902     1.256

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1319764    1288.832     1.259

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1324264    1293.227     1.263

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1329836    1298.66



>> ENERGY; Differences between the model's features and restraints:
Number of all residues in MODEL                   :        7
Number of all, selected real atoms                :       72      14
Number of all, selected pseudo atoms              :        0       0
Number of all static, selected restraints         :      352      97
COVALENT_CYS                                      :        F
NONBONDED_SEL_ATOMS                               :        1
Number of non-bonded pairs (excluding 1-2,1-3,1-4):       12
Dynamic pairs routine                             : 5, NATM_SEL x NATM double loop
Atomic shift for contacts update (UPDATE_DYNAMIC) :    0.390
LENNARD_JONES_SWITCH                              :    6.500   7.500
COULOMB_JONES_SWITCH                              :    6.500   7.500
RESIDUE_SPAN_RANGE                                :        0   99999
NLOGN_USE                                         :       15
CONTACT_SHELL                                     :    4.000
DYNAMI

openf___224_> Open           $(LIB)/par.lib

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1311816    1281.070     1.251

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1313416    1282.633     1.253

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1317396    1286.520     1.256

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1319396    1288.473     1.258

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1322396    1291.402     1.261

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1326896    1295.797     1.265

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1332468    1301.238     1.271

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1335268    1303.973     1.273

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1339468    1308.074     1.277

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:      1345768    1314.